In [ ]:
!pip install google-api-python-client

In [1]:
from secreto import API_KEY
import os
from googleapiclient.discovery import build
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
VideoId = "ZoIL42XmNlY" # 23-10-22

In [2]:
VideoId = "pMYLWBZlEKA" # 16-10-22 {'seis': 88, 'siete': 43, 'ocho': 0, 'nueve': 0}

In [7]:
def primera_pagina(VideoId):

    youtube = build(
        "youtube", "v3", developerKey = API_KEY)

    request = youtube.commentThreads().list(
        part="id, snippet, replies",
        videoId=VideoId,
        maxResults = 100
    )
    respuesta = request.execute()

    return respuesta

def siguiente_pagina(VideoId,nextPageToken):
    
    youtube = build(
        "youtube", "v3", developerKey = API_KEY)

    request = youtube.commentThreads().list(
        part = "id, snippet, replies",
        videoId = VideoId,
        pageToken = nextPageToken,
        maxResults = 100
    )
    response = request.execute()

    return response

def transformacion(respuesta):
    
    df = pd.DataFrame(columns=["autor","texto"])
    
    for n in range(0,len(respuesta["items"])):
        autor = respuesta["items"][n]["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"]
        texto = respuesta["items"][n]["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        df.loc[n,"autor"] = autor
        df.loc[n,"texto"] = texto

    return df


def crear_dataframe(VideoId):

    respuesta = primera_pagina(VideoId)
    
    df = transformacion(respuesta)
    
    while 'nextPageToken' in respuesta.keys():
        respuesta = siguiente_pagina(VideoId,respuesta['nextPageToken'])
        df2 = transformacion(respuesta)
        df = pd.concat([df,df2], axis=0)

    return df

def contar(VideoId):

    df = crear_dataframe(VideoId)

    seis = df["texto"].str.contains("4556").sum()
    siete = df["texto"].str.contains("4557").sum()
    ocho = df["texto"].str.contains("4558").sum()
    nueve = df["texto"].str.contains("4559").sum()
    diccionario = {
        "seis":seis,
        "siete":siete,
        "ocho":ocho,
        "nueve":nueve
        }
    return diccionario

In [8]:
resultados = contar(VideoId)

In [9]:
resultados

{'seis': 88, 'siete': 43, 'ocho': 0, 'nueve': 0}